In [0]:
def save_table(df, table_name, layer):
    (
        df.write.mode("append").format('delta')
        .option("mergeSchema", "true")    
        .option("delta.columnMapping.mode", "name")    
        .saveAsTable(f"{layer}_layer.{table_name}")
    )

In [0]:
def ingestao_dados(arquivos, tipo_arquivos):

    for file_path in arquivos:

        # Verifica se o tipo de arquivo é um CSV
        if(tipo_arquivos == 'csv'):

            # Obtém o nome da tabela a partir do nome do arquivo
            table_name = file_path.split('/')[5].replace('.csv', '')

            # Realiza a leitura dos dados em um dataframe com Spark
            dados = spark.read.options(
                header=True,
                inferSchema=True,
                delimiter=";"
            ).csv(file_path)

        else:

            # Obtém o nome da tabela a partir do nome do arquivo
            table_name = file_path.split('/')[5].replace('.json', '').replace('_json', '')

            # Realiza a leitura dos dados em um dataframe com Spark
            dados = spark.read.option("multiline", "true").json(file_path)  

            # Extrai os campos e registros a partir do JSON
            colunas = [campo["id"] for campo in dados.select("fields").first()["fields"]]
            tipos_colunas = [campo["type"] for campo in dados.select("fields").first()["fields"]]
            registros = dados.select("records").first()["records"]

            # Cria o dataframe com Spark a partir das informações importadas
            dados = spark.createDataFrame(registros, schema=colunas)

            # Variável auxiliar para mapear os tipos de dados
            mapa_tipos = {
                'text': 'string',
                'numeric': 'double',
                'int': 'int',
                'timestamp': 'timestamp'
            }

            # Ajusta os tipos de dados do dataframe criado a partir do arquivo JSON
            for coluna in colunas:

                indice = colunas.index(coluna)
                tipo_coluna = tipos_colunas[indice]
                tipo = mapa_tipos.get(tipo_coluna, 'string')  # default para 'string' se tipo não reconhecido
                if coluna in dados.columns:
                    dados = dados.withColumn(coluna, col(coluna).cast(tipo))

        # Leitura dos dados da tabela existente (caso já tinha sido salva anteriormente)
        dados_existentes = spark.read.table(f"bronze_layer.{table_name}")
        
        # Verificando se os dados importados já existem
        verifica_tabela = spark.catalog.tableExists(f"bronze_layer.{table_name}")

        # Se a tabela já existir, realiza filtragem no dataframe importado para inserir apenas os dados novos
        if verifica_tabela:         

            colunas = dados.columns

            condicoes = [dados[c] == dados_existentes[c] for c in colunas]

            df_filtrado = dados.join(dados_existentes, on=condicoes, how="left_anti")

            df_filtrado = df_filtrado.dropDuplicates()
            
            print('================================== CARGA DE DADOS ==================================')
            print('>>> Camada: Bronze')
            print(f'>>> Tipo do arquivo: {tipo_arquivos}')
            print(f'>>> Caminho Raw: {file_path}')
            print(f'>>> Tabela: {table_name}')
            print('>>> Executando carga de dados, aguarde...')

            try:        
                save_table(df_filtrado, table_name, 'bronze')
                print('=========================== CARGA FINALIZADA COM SUCESSO ===========================\n')
            except:
                print('============================= ERRO NA CARGA DE DADOS... ============================\n')    

        # Caso a tabela não exista, realiza a carga dos dados pela primeira vez
        else:
            
            print('================================== CARGA DE DADOS ==================================')
            print('>> Camada: Bronze')
            print(f'>> Tipo do arquivo: {tipo_arquivos}')
            print(f'>> Caminho Raw: {file_path}')
            print(f'>> Tabela: {table_name}')
            print('>>> Executando carga de dados, aguarde...')

            dados = dados.dropDuplicates()

            try:        
                save_table(dados, table_name, 'bronze')
                print('=========================== CARGA FINALIZADA COM SUCESSO ===========================\n')
            except:
                print('============================= ERRO NA CARGA DE DADOS... ============================\n')    
